In [1]:
# リスト 3.4.1 Wikipediaの日本百名湯記事をTF-IDFで分析（説明コメント付き）
# --------------------------------------------------------------------------------
# 目的:
#   ・以降の TF-IDF 分析の入力となる「Wikipedia 上に記事が存在する日本百名湯」の記事タイトル一覧。
#   ・この配列 spa_list をもとに、各記事の本文を収集→前処理→ベクトル化（TF-IDF）する。
#
# データ仕様/前提:
#   ・型: List[str]（純粋な記事タイトルの列）
#   ・表記: Wikipedia の**ページタイトル**に一致する表記を採用（例: 地名の重複は括弧つきで曖昧性解消）。
#       - 例: 「朝日温泉 (北海道)」「玉川温泉 (秋田県)」「二日市温泉 (筑紫野市)」
#   ・同義/別表記は**ここでは扱わない**（語彙統制は後段処理で対応）。タイトルは原則、Wikipedia 側の公式見出しを使用。
#
# 取得時の実務メモ:
#   ・Python の wikipedia / wikipediaapi ライブラリで記事本文を取得する際は、
#     `auto_suggest=False` を付けて**表記揺れで別ページに飛ばないように**するのが安全。
#   ・括弧つきタイトルは**完全一致**させないと別名・曖昧さ回避ページに誘導される可能性がある。
#   ・ページが存在しない/移動済みの場合は 404/Redirect/Disambiguation をハンドリングすること。
#
# 後工程（TF-IDF）での注意:
#   ・日本語はスペースで分かち書きされないため、ベクトル化には
#       (A) 形態素解析（MeCab/Janome + TfidfVectorizer(tokenizer=...)）
#       (B) 文字 n-gram（例: analyzer="char_wb", ngram_range=(2,4)）
#     のいずれかを選ぶ。学術用途は (A) 推奨だがセットアップ容易性は (B) が高い。
#   ・本文取得後は、注記/脚注/テンプレ/目次の除去、記号正規化（NFKC）、全半角統一などの前処理が精度に寄与。
#
# 品質チェックのヒント（必要に応じて実行）:
#   # 重複検出: assert len(spa_list) == len(set(spa_list))
#   # 存在確認: 404/曖昧さ回避を検知し、欠損リストを別途保存しておくと再現性が上がる。
# --------------------------------------------------------------------------------

# 日本百名湯のうち、Wikipedia に記事のある温泉のリスト（記事タイトル完全一致）
spa_list = [
    "菅野温泉",
    "養老牛温泉",
    "定山渓温泉",
    "登別温泉",
    "洞爺湖温泉",
    "ニセコ温泉郷",
    "朝日温泉 (北海道)",
    "酸ヶ湯温泉",
    "蔦温泉",
    "花巻南温泉峡",
    "夏油温泉",
    "須川高原温泉",
    "鳴子温泉郷",
    "遠刈田温泉",
    "峩々温泉",
    "乳頭温泉郷",
    "後生掛温泉",
    "玉川温泉 (秋田県)",
    "秋ノ宮温泉郷",
    "銀山温泉",
    "瀬見温泉",
    "赤倉温泉 (山形県)",
    "東山温泉",
    "飯坂温泉",
    "二岐温泉",
    "那須温泉郷",
    "塩原温泉郷",
    "鬼怒川温泉",
    "奥鬼怒温泉郷",
    "草津温泉",
    "伊香保温泉",
    "四万温泉",
    "法師温泉",
    "箱根温泉",
    "湯河原温泉",
    "越後湯沢温泉",
    "松之山温泉",
    "大牧温泉",
    "山中温泉",
    "山代温泉",
    "粟津温泉",
    "奈良田温泉",
    "西山温泉 (山梨県)",
    "野沢温泉",
    "湯田中温泉",
    "別所温泉",
    "中房温泉",
    "白骨温泉",
    "小谷温泉",
    "下呂温泉",
    "福地温泉",
    "熱海温泉",
    "伊東温泉",
    "修善寺温泉",
    "湯谷温泉 (愛知県)",
    "榊原温泉",
    "木津温泉",
    "有馬温泉",
    "城崎温泉",
    "湯村温泉 (兵庫県)",
    "十津川温泉",
    "南紀白浜温泉",
    "南紀勝浦温泉",
    "湯の峰温泉",
    "龍神温泉",
    "奥津温泉",
    "湯原温泉",
    "三朝温泉",
    "岩井温泉",
    "関金温泉",
    "玉造温泉",
    "有福温泉",
    "温泉津温泉",
    "湯田温泉",
    "長門湯本温泉",
    "祖谷温泉",
    "道後温泉",
    "二日市温泉 (筑紫野市)",
    "嬉野温泉",
    "武雄温泉",
    "雲仙温泉",
    "小浜温泉",
    "黒川温泉",
    "地獄温泉",
    "垂玉温泉",
    "杖立温泉",
    "日奈久温泉",
    "鉄輪温泉",
    "明礬温泉",
    "由布院温泉",
    "川底温泉",
    "長湯温泉",
    "京町温泉",
    "指宿温泉",
    "霧島温泉郷",
    "新川渓谷温泉郷",
    "栗野岳温泉",
]

# （参考）後続処理の典型フロー（ここでは実行しない・骨子のみ）
# from wikipedia import summary, set_lang
# set_lang("ja")
# texts = []
# for title in spa_list:
#     try:
#         texts.append(summary(title, auto_suggest=False))
#     except Exception as e:
#         # DisambiguationError/PageError を拾って要確認リストへ回す
#         pass
# # 形態素解析 or 文字 n-gram 前処理 → TfidfVectorizer.fit_transform(texts)

In [2]:
# -----------------------------------------------------------------------------
# Wikipediaの記事の読み取り（日本百名湯）— 説明コメント付き
# 目的:
#   - spa_list に含まれる各温泉の Wikipedia 記事本文（.content）を取得し、content_list に格納する。
# 前提:
#   - 2.1節で定義済みの spa_list（ページタイトルの配列）が存在すること。
#   - ネットワーク環境があること（wikipedia パッケージはオンラインで Wikipedia にアクセスする）。
# 使用ライブラリ:
#   - wikipedia（PyPI: wikipedia）：MediaWiki API を叩く薄いラッパ。簡便だが曖昧さ処理の例外が発生しやすい。
# 設計メモ（理論）:
#   - set_lang("ja") により検索対象を日本語版 Wikipedia に固定する（多言語ページの誤ヒットを避ける）。
#   - page(title, auto_suggest=False) は「完全一致」志向。自動補完で別ページに飛ばない利点がある一方、
#     タイトルの誤記や表記揺れがあると PageError/DisambiguationError を起こしやすい。
#   - .content はページ本文のテキスト（マークアップ除去済みのことが多いが、節タイトル等は含まれる）。
#   - 収集後は前処理（正規化・ノイズ除去）→ ベクトル化（例: TF-IDF）へ進む想定。
# 例外と実務上の注意:
#   - wikipedia.exceptions.DisambiguationError：曖昧さ回避ページに当たった場合に発生（例: 同名温泉が複数）。
#   - wikipedia.exceptions.PageError：タイトルに対応するページが存在しない場合に発生。
#   - 連続アクセスが多い場合、礼儀として待機を入れる（time.sleep）。API 側レート制限にも配慮。
#   - 取得漏れを把握するために、成功/失敗のタイトルを別リストに蓄積すると再現性が上がる。
# -----------------------------------------------------------------------------

# wikipedia パッケージの読み込みと日本語版の指定
import wikipedia

wikipedia.set_lang("ja")  # 以降の page/summary は日本語版 Wikipedia を対象にする

# 取得した本文を格納するリスト
content_list = []

# 各温泉タイトルに対して Wikipedia ページ本文を取得
for spa in spa_list:
    print(spa)  # 進捗ログ: 現在処理中のページタイトルを表示
    # auto_suggest=False:
    #   - 自動補完を無効化して、与えたタイトルにできるだけ厳密に一致するページを取得する。
    #   - 曖昧さ回避や誤補完による誤取得を避ける狙い。
    content = wikipedia.page(spa, auto_suggest=False).content
    content_list.append(content)

# -----------------------------------------------------------------------------
# （発展: 例外処理・レート制御を入れたい場合の雛形。必要に応じて使用）
# import time
# from wikipedia.exceptions import DisambiguationError, PageError
# content_list, failed = [], []
# for spa in spa_list:
#     try:
#         print(spa)
#         txt = wikipedia.page(spa, auto_suggest=False).content
#         content_list.append(txt)
#         time.sleep(0.5)  # マナーとしてのウェイト（環境に応じて調整）
#     except DisambiguationError as e:
#         # e.options に候補リストが入る。必要なら手動で最適候補を選ぶ処理を追加。
#         failed.append({"title": spa, "reason": "disambiguation", "options": e.options[:5]})
#     except PageError:
#         failed.append({"title": spa, "reason": "page_not_found"})
# # 収集漏れ確認:
# # print(f"成功: {len(content_list)} 件 / 失敗: {len(failed)} 件"); print(failed)
# -----------------------------------------------------------------------------

菅野温泉
養老牛温泉
定山渓温泉
登別温泉
洞爺湖温泉
ニセコ温泉郷
朝日温泉 (北海道)
酸ヶ湯温泉
蔦温泉
花巻南温泉峡
夏油温泉
須川高原温泉
鳴子温泉郷
遠刈田温泉
峩々温泉
乳頭温泉郷
後生掛温泉
玉川温泉 (秋田県)
秋ノ宮温泉郷
銀山温泉
瀬見温泉
赤倉温泉 (山形県)
東山温泉
飯坂温泉
二岐温泉
那須温泉郷
塩原温泉郷
鬼怒川温泉
奥鬼怒温泉郷
草津温泉
伊香保温泉
四万温泉
法師温泉
箱根温泉
湯河原温泉
越後湯沢温泉
松之山温泉
大牧温泉
山中温泉
山代温泉
粟津温泉
奈良田温泉
西山温泉 (山梨県)
野沢温泉
湯田中温泉
別所温泉
中房温泉
白骨温泉
小谷温泉
下呂温泉
福地温泉
熱海温泉
伊東温泉
修善寺温泉
湯谷温泉 (愛知県)
榊原温泉
木津温泉
有馬温泉
城崎温泉
湯村温泉 (兵庫県)
十津川温泉
南紀白浜温泉
南紀勝浦温泉
湯の峰温泉
龍神温泉
奥津温泉
湯原温泉
三朝温泉
岩井温泉
関金温泉
玉造温泉
有福温泉
温泉津温泉
湯田温泉
長門湯本温泉
祖谷温泉
道後温泉
二日市温泉 (筑紫野市)
嬉野温泉
武雄温泉
雲仙温泉
小浜温泉
黒川温泉
地獄温泉
垂玉温泉
杖立温泉
日奈久温泉
鉄輪温泉
明礬温泉
由布院温泉
川底温泉
長湯温泉
京町温泉
指宿温泉
霧島温泉郷
新川渓谷温泉郷
栗野岳温泉


In [3]:
# 形態素解析
# 2.2節参照

from janome.tokenizer import Tokenizer

# Tokenizer インスタンスの生成
# - Janome は純粋 Python 実装の日本語形態素解析器（追加インストールのみで動作）
# - 毎回生成するとコストが高いため、モジュールレベルで 1 度だけ生成して使い回す
t = Tokenizer()


# 形態素解析関数の定義
def tokenize(text):
    """
    与えられた日本語テキストを Janome で形態素解析し、
    「名詞」および「形容詞」に該当する語の原形（base_form）のみを配列で返す。

    返り値:
        List[str]: 名詞・形容詞の原形のみからなるトークン列

    実装メモ:
    - token.part_of_speech は「品詞,品詞細分類1,品詞細分類2,品詞細分類3」のカンマ区切り文字列。
      先頭要素（品詞大分類）が '名詞' または '形容詞' のものだけを採用する。
    - token.base_form は語の原形。名詞は多くの場合 surface と同一、形容詞は原形化される（例: 「高かった」→「高い」）。
    - 未知語では base_form が '*' となる場合がある（この関数ではそのまま返る点に注意）。
      必要なら '*' を surface で置き換える処理を追加してもよい（下に例をコメントで記載）。
    - 動詞も分析対象に含めたい場合は、条件の配列に '動詞' を追加するだけでよい。
    """
    return [
        token.base_form
        for token in t.tokenize(text)
        # 品詞大分類（先頭ラベル）でフィルタ
        if token.part_of_speech.split(",")[0] in ["名詞", "形容詞"]
    ]


# --- 拡張の例（必要なら使用） -----------------------------------------------
# def tokenize(text):
#     tokens = []
#     for tok in t.tokenize(text):
#         pos = tok.part_of_speech.split(',')[0]
#         if pos in ['名詞', '形容詞']:
#             base = tok.base_form if tok.base_form != '*' else tok.surface  # 未知語を表層形にフォールバック
#             tokens.append(base)
#     return tokens
# ------------------------------------------------------------------------------

In [4]:
# wikipedia記事を名詞と形容詞のみとし、ブランクで分かち書き
# 2.2節参照（tokenize: 名詞・形容詞のみを原形で返す関数を使用）

# 目的:
#   ・各 Wikipedia 記事本文（content）から、解析対象語（名詞・形容詞）のみを抽出し、
#     空白区切り（スペース区切り）の 1 行テキストへ変換してコーパス words_list に格納する。
#   ・以降の TF-IDF（または CountVectorizer）に、そのまま渡せる形を用意する。
#
# 理論/設計メモ:
#   ・ベクトル化器（TfidfVectorizer など）がデフォルトで「空白区切り」を単語境界とみなす前提で整形。
#   ・日本語は本来スペースで分かち書きされないため、先に形態素解析で単語列にしてから ' '.join(...) する。
#   ・Wikipedia の heading マーク「== 見出し ==」に含まれる "==" は、そもそも品詞が記号のため
#     tokenize（名詞/形容詞抽出）では落ちるが、念のため '==' を除去する処理を残している。
#     （厳密に見出し行全体を落とすには、事前に正規表現で `==...==` の行を削除するのが堅牢）
#
# 実務上の注意:
#   ・content が None/空文字のケースや、未知語が base_form='*' となるケースに留意。
#     必要なら '*' を surface へフォールバックする tokenize 拡張を検討。
#   ・後続の TF-IDF で記号・数字を落としたい場合は、ここで正規化/置換を行うと一貫性が出る。
#   ・spa_list と content_list のインデックス対応が崩れないように、例外処理を入れる際は
#     失敗要素を別途記録する（タイトルと一緒に）など再現性の担保を。
#
# 例の拡張（必要に応じて使用・下にコメントで雛形あり）:
#   - 正規化: NFKC、全/半角統一、改行→スペース、連続空白の縮約
#   - heading 行の完全除去: re.sub(r"^==+.*?==+$", "", text, flags=re.MULTILINE)
#   - 記号/数字の除去: re.sub(r"[0-9０-９]+", "0", text) など

words_list = []
for content in content_list:
    # 1) 形態素解析で名詞・形容詞の原形シーケンスに変換
    tokens = tokenize(content)  # 例: ["温泉","湧出","豊富","名所", ...]
    # 2) 空白区切りの 1 行テキストへ変換（ベクトル化器に素直に渡せる形）
    words = " ".join(tokens)
    # 3) 見出しマークの保険的な除去（多くの場合、tokens に "==" は含まれないため冗長だが無害）
    words = words.replace("==", "")
    # 4) 必要なら追加の軽量正規化（任意・例）
    #    - 連続空白の縮約:
    # import re
    # words = re.sub(r"\s+", " ", words).strip()
    #    - 数字の正規化（全部 0 へ統一など）:
    # words = re.sub(r"[0-9０-９]+", "0", words)

    words_list.append(words)

# --- 参考: 見出し行を事前に丸ごと落としたい場合の雛形（tokenize 前に適用） ------------
# import re
# words_list = []
# for content in content_list:
#     cleaned = re.sub(r"^==+.*?==+$", "", content, flags=re.MULTILINE)  # 見出し行の除去
#     tokens = tokenize(cleaned)
#     words = ' '.join(tokens)
#     words_list.append(re.sub(r"\s+", " ", words).strip())
# ----------------------------------------------------------------------------------------------

In [5]:
# リスト 3.4.2
# TF-IDF分析の実施（詳細コメント付き）

# ライブラリのインポート
# - TfidfVectorizer: 文書集合から TF-IDF（用語頻度×逆文書頻度）によるベクトル表現を生成する。
from sklearn.feature_extraction.text import TfidfVectorizer

# ベクトライザの初期化
# -----------------------------------------------------------------------------
# 前処理前提:
#   ・すでに words_list は「形態素解析済みのトークンを空白で連結した文字列」の配列である
#     （例: "温泉 豊富 名所 ..."}）。
# 設計ポイント:
#   ・token_pattern の既定は r'(?u)\b\w\w+\b'（= 2 文字以上のトークンのみ有効）。
#     日本語では 1 文字名詞（例: 「湯」「滝」など）も扱いたい場合があるため、
#     ここでは 1 文字以上を許す r'(?u)\b\w+\b' に変更している。
#   ・min_df=1: 1 文書にしか出ない語も特徴に残す（学習用の可観測性重視）。
#   ・max_df=50: 51 文書以上に出現する汎用語を落とす（df > 50 を切る上限）。
#     ※ 文書数（= len(words_list)）に応じて適宜調整。割合指定（例: max_df=0.5）も可。
#   ・norm='l2': ベクトルを L2 正規化（各文書ベクトルの長さを 1 に）→ 文書長の影響を抑える。
vectorizer = TfidfVectorizer(
    analyzer="word",
    token_pattern=r"(?u)\b\w+\b",  # 1 文字語も採用（既定は 2 文字以上）
    min_df=1,
    max_df=50,
    norm="l2",
    use_idf=True,  # 逆文書頻度を掛ける
    smooth_idf=True,  # idf のスムージング（ゼロ回避）
    sublinear_tf=False,  # TF のサブリニア（log(1+tf)）はここでは無効（必要に応じて True）
)

# フィーチャーベクトルの生成（疎行列; shape = [文書数, 語彙数]）
# - fit: 語彙（ボキャブラリ）を学習
# - transform: TF-IDF を計算
# - fit_transform: 上記を一括実行
features = vectorizer.fit_transform(words_list)

# 特徴語（語彙リスト）の抽出
# - scikit-learn >= 1.0: get_feature_names_out()
# - 旧バージョン互換のため try/except で両対応
try:
    terms = vectorizer.get_feature_names_out()
except AttributeError:
    terms = vectorizer.get_feature_names()

# 疎行列を TF-IDF の密行列（numpy.ndarray）へ変換
# - 大規模コーパスではメモリ使用量が跳ね上がるため、解析・可視化などの必要時のみ実施する。
tfidfs = features.toarray()

# 参考: 形状や一部の確認（必要ならコメント解除）
# print(f"docs={features.shape[0]}, vocab={features.shape[1]}")
# print("sample terms:", terms[:20])
# print("TF-IDF[0, :5]:", tfidfs[0, :5])

In [6]:
# リスト 3.4.3 温泉毎の特徴語の表示（詳細コメント付き）

import numpy as np


def extract_feature_words(terms, tfidfs, i, n):
    """
    i 番目の文書（温泉記事）について、TF-IDF 値が高い上位 n 語を返す。

    引数:
        terms : array-like (語彙リスト; shape = [V])
            TfidfVectorizer から得た語彙（特徴量名）。インデックスは列次元に対応。
        tfidfs: np.ndarray (TF-IDF 行列; shape = [D, V])
            fit_transform(...).toarray() 等で得た密行列（D=文書数, V=語彙数）。
        i     : int
            対象とする文書インデックス（0 <= i < D）。
        n     : int
            返す上位語数（n > V の場合は V に丸める）。

    戻り値:
        List[str]: 文書 i における TF-IDF が高い順の上位 n 語（語形は terms の表記）

    実装方針 / 理論メモ:
        - TF-IDF は「その文書で相対的に重要な語」を浮かび上がらせる指標。
          * TF（Term Frequency）: 文書内頻度
          * IDF（Inverse Document Frequency）: 文書間の一般性の低さ（希少性）
        - 上位抽出は、語彙サイズ V が大きい場合に O(V log V) の完全ソートは重い。
          ここでは np.argpartition を用いて O(V) で上位 n の候補を抽出し、
          その部分だけ降順に並べ替えることで計算量を削減する。
        - TF-IDF が同点の語の順序は未定義（語彙インデックス順になる可能性）。
    """
    # 型/次元の基本チェック（学習コードの可観測性向上のため）
    tfidfs = np.asarray(tfidfs)
    D, V = tfidfs.shape
    if not (0 <= i < D):
        raise IndexError(f"文書インデックス i={i} が範囲外です（0 <= i < {D}）。")
    if V == 0:
        return []

    # 要求語数を語彙数に丸める
    n = int(n)
    n = max(0, min(n, V))
    if n == 0:
        return []

    # i 番目の文書ベクトルを取得
    tfidf_row = tfidfs[i]

    # 上位 n の候補インデックスを argpartition で取得（ここでは n 個の“最大要素”集合）
    # 注意: argpartition は順序を保証しないため、後段で降順にソートし直す
    candidate_idx = np.argpartition(tfidf_row, -n)[-n:]

    # 候補を TF-IDF 降順に整列
    sorted_local = candidate_idx[np.argsort(tfidf_row[candidate_idx])[::-1]]

    # 語彙へマッピング
    return [terms[idx] for idx in sorted_local]


# --- 結果の出力 ---------------------------------------------------------------
# 先頭 10 件の温泉について、上位 10 語を表示。
# ・words_list / terms / tfidfs / spa_list が事前に定義されている前提（前リスト参照）。
# ・インデックス対応の安全化のため、available_docs = min(10, len(spa_list), tfidfs.shape[0]) とする。
available_docs = min(10, len(spa_list), tfidfs.shape[0])

for i in range(available_docs):
    print("【" + spa_list[i] + "】")
    top_words = extract_feature_words(terms, tfidfs, i, 10)
    # 語のみを空白区切りで出力（元コード互換）
    print(" ".join(top_words))

# --- 参考: スコア付きで見たい場合（必要時のみコメント解除） --------------------
# def extract_feature_words_with_scores(terms, tfidfs, i, n):
#     tfidfs = np.asarray(tfidfs)
#     D, V = tfidfs.shape
#     if not (0 <= i < D): raise IndexError("i out of range")
#     n = max(0, min(int(n), V))
#     if n == 0: return []
#     row = tfidfs[i]
#     cand = np.argpartition(row, -n)[-n:]
#     order = cand[np.argsort(row[cand])[::-1]]
#     return [(terms[j], float(row[j])) for j in order]
#
# for i in range(available_docs):
#     print('【' + spa_list[i] + '】')
#     for w, s in extract_feature_words_with_scores(terms, tfidfs, i, 10):
#         print(f"{w}:{s:.3f}", end=' ')
#     print()

【菅野温泉】
然別 菅野 かん 峡 再開 食塩 重曹 湯舟 鹿追 営業
【養老牛温泉】
養老牛 坂本 裏 標津 開業 まつ 西村 アイヌ 藤 堀口
【定山渓温泉】
定山渓 かっぱ 札幌 山 橋 淵 小樽 完成 定 北海道
【登別温泉】
登別 登別温泉 地獄谷 地獄 北海道 大湯沼 滝本 岡田 大正 日和山
【洞爺湖温泉】
洞爺湖温泉 洞爺湖 洞爺 虻田 有珠山 壮瞥 北海道 湖 とうや ジオパーク
【ニセコ温泉郷】
ニセコ パス 名人 温泉郷 スタンプ 蘭越 ニセコアンヌプリ 倶知安 贈呈 北海道
【朝日温泉 (北海道)】
岩内 朝日 土砂 災害 休業 雷電 ユウ ナイ川 内川 2010
【酸ヶ湯温泉】
八甲田山 熱 青森 植物 八甲田 千 気候 時 混浴 午前
【蔦温泉】
蔦 十和田 沼 猪木 野鳥 コース 森 要塞 アントニオ 東北
【花巻南温泉峡】
花巻 鉛 峡 花巻温泉 豊沢川 はな 戸平 東和 松倉 金矢
